### 네이버 뉴스 섹션별 TOPIC 크롤링
**정치, 경제 ,사회, 생활/문화, IT/과학, 세계**


In [1]:
from urllib.request import urlopen
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# 02_0에서 저장한 섹션별 link파일 불러오기
#
df_menu=pd.read_csv('crawl_data/naver_news_section.csv',index_col=0)
df_menu.head()

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...


#### 경제 섹션을 예시 코드로 작성

In [3]:
df_menu['link'][2]

'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101'

In [4]:
# 경제 섹션 소스 요청
url=df_menu['link'][2]
res=requests.get(url) # 요청 후 응답객체 반환
res
# ConnectionError: ('Connection aborted.', 
# ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', 
# None, 10054, None))
# 원격 봇이라고 생각하고 연결 종료
# res

# urlopen() 함수는  진행됨
# res=urlopen(url)
# res.read()

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

- 위 코드처럼 자동화 봇으로 보고 연결을 끊는 경우 해결방법
    - 서버에 요청 시, header를 구성해 추가해 줌
    - 즉, bot이 아님을 증명
        - 브라우저에서 해당 사이트 접속 시 생성하는 헤더를
        - 개발자 도구로 확인(브라우저 정보 : User-Agent)

In [6]:
# 개발자환경-networks-main.naver~~-headers-User-Agent
# 소스 가져오기
url=df_menu['link'][2]
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
res=requests.get(url,headers=headers)

In [7]:
html=res.text

In [8]:
url=df_menu['link'][2]
html=urlopen(url)

In [9]:
html

In [10]:
bs_obj=BeautifulSoup(html,'html.parser')

In [11]:
bs_obj


<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh">
<meta content="width=1106" name="viewport">
<meta content="경제 : 네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://news.naver.com/main/main.naver?mode=LSD&amp;mid=shm&amp;sid1=101" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" property="og:image">
<meta content="증권, 금융, 부동산, 기업, 국제 등 경제 분야 뉴스 제공" property="og:description"/>
<meta content="네이버" property="og:article:author">
<meta content="summary" name="twitter:card"/>
<meta content="경제 : 네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" name="tw

- 네이버 뉴스 섹션을 확인하면 각 섹션별 topic이 있고, topic중 헤드라인이 노출
- 섹션에서 수집하는 것 보다 토픽페이지로 이동해서 토픽기사 전체를 크롤링하면 더 많은 기사를 추출할 수 있음
- 섹션 페이지에서는 토픽의 title과 link를 크롤링
- 정치 섹션은 구성이 다르므로 수정해서 사용

In [13]:
# 토픽 추출
# #main_content > div > div,persist
# h2태그의 class::cluster_head_topic
topic_list=bs_obj.findAll('h2',{'class':'cluster_head_topic'})
len(topic_list)

13

In [14]:
topic_list

[<h2 class="cluster_head_topic">
 <a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202207311550_00000316&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=031&amp;aid=0000689055">
 <span class="cluster_head_sub_topic">금감원 명령휴가제 강화 추진 </span><span class="cluster_head_sub_topic">은행 잇단 횡령사고에 </span>
 </a>
 </h2>,
 <h2 class="cluster_head_topic">
 <a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202208011210_00000016&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=421&amp;aid=0006252486">
 <span class="cluster_head_sub_topic">6월 온라인쇼핑 거래액 16조7806억 </span><span class="cluster_head_sub_topic">여행·교통 수요 폭발 </span>
 </a>
 </h2>,
 <h2 class="cluster_head_topic">
 <a class="nclicks(cls_eco.clstitle)" href="/main/clusterArticles.naver?id=c_202208010910_00000005&amp;mode=LSD&amp;mid=shm&amp;sid1=101&amp;oid=025&amp;aid=0003213647">
 <span class="cluster_head_sub_topic">7월 무역수지 46.7억달러 적자 </span><span class="cluster_head_sub_topic">14년만에 넉달 연속 </span>


In [15]:
topic_list[0].find('a')
topic_list[0].find('a')['href']
topic_list[0].find('a').text.replace('\n',"")

'금감원 명령휴가제 강화 추진 은행 잇단 횡령사고에 '

In [16]:
# 전체 토픽 title과 link 출력
for topic in topic_list:
    try:
        a_tag=topic.find('a')
        print(a_tag.text.replace('\n',''))
        print(a_tag['href'])
    except:
        print('에러발생')
# 마지막 부분에서 a태그 없는요소(빈 item)가 있어서 에러 발생
# 예외 처리 진행

금감원 명령휴가제 강화 추진 은행 잇단 횡령사고에 
/main/clusterArticles.naver?id=c_202207311550_00000316&mode=LSD&mid=shm&sid1=101&oid=031&aid=0000689055
6월 온라인쇼핑 거래액 16조7806억 여행·교통 수요 폭발 
/main/clusterArticles.naver?id=c_202208011210_00000016&mode=LSD&mid=shm&sid1=101&oid=421&aid=0006252486
7월 무역수지 46.7억달러 적자 14년만에 넉달 연속 
/main/clusterArticles.naver?id=c_202208010910_00000005&mode=LSD&mid=shm&sid1=101&oid=025&aid=0003213647
유류세 추가 인하 효과 휘발유 평균가격 1800원대 진입 
/main/clusterArticles.naver?id=c_202207300610_00000025&mode=LSD&mid=shm&sid1=101&oid=028&aid=0002600676
"전기차 배터리 구독서비스" 허용 4530만원 니로 전기차1430만원에 산다 
/main/clusterArticles.naver?id=c_202208010610_00000012&mode=LSD&mid=shm&sid1=101&oid=025&aid=0003213804
공매도 줄고 외국인 돌아오고 올 들어 최저 
/main/clusterArticles.naver?id=c_202207242350_00000725&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004998695
상위 1%만 낸다 부부공동 1주택 종부세 기준선 
/main/clusterArticles.naver?id=c_202208010320_00000266&mode=LSD&mid=shm&sid1=101&oid=011&aid=0004082538
이창용 한은 총재 "기준금리 0.25%p씩 인상이 적절" 
/main/cluster

- base_url : http://news.naver.com
- sub_url : /main/clusterArticles.naverid=c_202207261210_00000008&mode=LSD&
    mid=shm&sid1=101&oid=011&aid=0004080341
- 수집된 url은 기본 url이 빠져 있으므로 기본 url을 첨부하여 수집해야 함

In [17]:
# link에 기본 url 추가
base_url='http://news.naver.com'
# 전체 토픽 title과 link 출력
for topic in topic_list:
    try:
        a_tag=topic.find('a')
        print(a_tag.text.replace('\n',''))
        print(base_url+a_tag['href'])
    except:
        print('에러발생')

금감원 명령휴가제 강화 추진 은행 잇단 횡령사고에 
http://news.naver.com/main/clusterArticles.naver?id=c_202207311550_00000316&mode=LSD&mid=shm&sid1=101&oid=031&aid=0000689055
6월 온라인쇼핑 거래액 16조7806억 여행·교통 수요 폭발 
http://news.naver.com/main/clusterArticles.naver?id=c_202208011210_00000016&mode=LSD&mid=shm&sid1=101&oid=421&aid=0006252486
7월 무역수지 46.7억달러 적자 14년만에 넉달 연속 
http://news.naver.com/main/clusterArticles.naver?id=c_202208010910_00000005&mode=LSD&mid=shm&sid1=101&oid=025&aid=0003213647
유류세 추가 인하 효과 휘발유 평균가격 1800원대 진입 
http://news.naver.com/main/clusterArticles.naver?id=c_202207300610_00000025&mode=LSD&mid=shm&sid1=101&oid=028&aid=0002600676
"전기차 배터리 구독서비스" 허용 4530만원 니로 전기차1430만원에 산다 
http://news.naver.com/main/clusterArticles.naver?id=c_202208010610_00000012&mode=LSD&mid=shm&sid1=101&oid=025&aid=0003213804
공매도 줄고 외국인 돌아오고 올 들어 최저 
http://news.naver.com/main/clusterArticles.naver?id=c_202207242350_00000725&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004998695
상위 1%만 낸다 부부공동 1주택 종부세 기준선 
http://news.naver.com/ma

### 수집한 데이터를 데이터프레임으로 생성

In [18]:
# 수집 내용 list에 저장
topic_title=[]
topic_link=[]

In [19]:
# link에 기본 url 추가
base_url='http://news.naver.com'
# 전체 토픽 title과 link 출력
for topic in topic_list:
    try:
        a_tag=topic.find('a')
        topic_title.append(a_tag.text.replace('\n',''))
        topic_link.append(base_url+a_tag['href'])
    except:
        print('에러발생')

에러발생


In [20]:
topic_title
topic_link

['http://news.naver.com/main/clusterArticles.naver?id=c_202207311550_00000316&mode=LSD&mid=shm&sid1=101&oid=031&aid=0000689055',
 'http://news.naver.com/main/clusterArticles.naver?id=c_202208011210_00000016&mode=LSD&mid=shm&sid1=101&oid=421&aid=0006252486',
 'http://news.naver.com/main/clusterArticles.naver?id=c_202208010910_00000005&mode=LSD&mid=shm&sid1=101&oid=025&aid=0003213647',
 'http://news.naver.com/main/clusterArticles.naver?id=c_202207300610_00000025&mode=LSD&mid=shm&sid1=101&oid=028&aid=0002600676',
 'http://news.naver.com/main/clusterArticles.naver?id=c_202208010610_00000012&mode=LSD&mid=shm&sid1=101&oid=025&aid=0003213804',
 'http://news.naver.com/main/clusterArticles.naver?id=c_202207242350_00000725&mode=LSD&mid=shm&sid1=101&oid=009&aid=0004998695',
 'http://news.naver.com/main/clusterArticles.naver?id=c_202208010320_00000266&mode=LSD&mid=shm&sid1=101&oid=011&aid=0004082538',
 'http://news.naver.com/main/clusterArticles.naver?id=c_202207310520_00000624&mode=LSD&mid=shm&si

In [21]:
topic_df=pd.DataFrame({'contents':topic_title,'URL':topic_link})
topic_df.head()

,contents,URL
0,금감원 명령휴가제 강화 추진 은행 잇단 횡령사고에,http://news.naver.com/main/clusterArticles.nav...
1,6월 온라인쇼핑 거래액 16조7806억 여행·교통 수요 폭발,http://news.naver.com/main/clusterArticles.nav...
2,7월 무역수지 46.7억달러 적자 14년만에 넉달 연속,http://news.naver.com/main/clusterArticles.nav...
3,유류세 추가 인하 효과 휘발유 평균가격 1800원대 진입,http://news.naver.com/main/clusterArticles.nav...
4,"""전기차 배터리 구독서비스"" 허용 4530만원 니로 전기차1430만원에 산다",http://news.naver.com/main/clusterArticles.nav...


### 위 내용을 결합해서 함수로 구성
- 다른 섹션에도 적용하기 위해서
- 함수명 : get_topic(url)
- 반환 : 섹션별 수집 데이터를 dict로 구성 후 반환

In [22]:
# 함수구성
def get_topic(url,section):
    # 수집내용 list에 저장
    topic_title=[]
    topic_link=[]
    
    # 소스 요청해서 가져오기
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
    res=requests.get(url,headers=headers)
    
    # 파서기 생성(bs4 객체 생성)
    html=res.text
    bs_obj=BeautifulSoup(html,'html.parser')
    
    # 토픽 추출
    # h2태그의 class::cluster_head_topic
    topic_list=bs_obj.findAll('h2',{'class':'cluster_head_topic'})
    
    # 최종 수집항목을 list에 저장
    # link에 기본 url 추가
    # link에 기본 url 추가
    base_url='http://news.naver.com'
    # 전체 토픽 title과 link 출력
    for topic in topic_list:
        try:
            a_tag=topic.find('a')
            topic_title.append(a_tag.text.replace('\n',''))
            topic_link.append(base_url+a_tag['href'])
        except:
            print('에러발생')
    return({'topic':topic_title,'URL':topic_link,'section':section})

In [23]:
# 함수 테스트
get_topic(url,'경제')

에러발생


{'topic': ['7월 무역수지 46.7억달러 적자 14년만에 넉달 연속 ',
  'SK하이닉스 차세대 메모리 CXL 개발 ',
  '전남 신안 자은도 국토부 지역개발사업 11건 선정 ',
  '추경호 부자감세 아냐 법인세 인하 비정상 포기하라 ',
  '금감원 명령휴가제 강화 추진 은행 잇단 횡령사고에 ',
  '유류세 추가 인하 효과 휘발유 평균가격 1800원대 진입 ',
  'NH투자증권 지속가능통합보고서 발간 ESG 우리가 으뜸 ',
  '원희룡 250만호+α 10일 전후 발표 ',
  '상상인저축은행 정기예금 금리 인상 최고 연 3.81% ',
  '"전기차 배터리 구독서비스" 허용 4530만원 니로 전기차1430만원에 산다 ',
  '"대형마트 의무휴업 폐지" 국민투표 1위 국민제안 무효 아쉽지만 ',
  '이창용 한은 총재 "기준금리 0.25%p씩 인상이 적절" '],
 'URL': ['http://news.naver.com/main/clusterArticles.naver?id=c_202208010910_00000005&mode=LSD&mid=shm&sid1=101&oid=018&aid=0005281508',
  'http://news.naver.com/main/clusterArticles.naver?id=c_202208011000_00000003&mode=LSD&mid=shm&sid1=101&oid=015&aid=0004731506',
  'http://news.naver.com/main/clusterArticles.naver?id=c_202208011110_00000007&mode=LSD&mid=shm&sid1=101&oid=014&aid=0004876338',
  'http://news.naver.com/main/clusterArticles.naver?id=c_202208011230_00000001&mode=LSD&mid=shm&sid1=101&oid=015&aid=0004731418',
  'http://news.naver.com/mai

In [24]:
# 02_0에서 저장한 섹션별 url파일 가져오기
df_menu=pd.read_csv('crawl_data/naver_news_section.csv',index_col=0)
df_menu.head(8)

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
5,IT/과학,https://news.naver.com/main/main.naver?mode=LS...
6,세계,https://news.naver.com/main/main.naver?mode=LS...
7,랭킹,https://news.naver.com/main/ranking/popularDay...


In [25]:
# 함수 test
pd.DataFrame(get_topic(df_menu['link'][3],df_menu['section'][3]))

에러발생


,topic,URL,section
0,경품 걸고 장기자랑 불법접객행위 제주 게스트하우스 적발,http://news.naver.com/main/clusterArticles.nav...,사회
1,"대법 첫 판단 ""휴대전화 압수영장으로 클라우드까지는 안돼""",http://news.naver.com/main/clusterArticles.nav...,사회
2,"""탈북어민 강제 북송"" 의혹 서훈 귀국 ""일정 따라 조사""",http://news.naver.com/main/clusterArticles.nav...,사회
3,"백신 부작용 실마리 발견 ""간 기능 이상"" 국내 첫 확인",http://news.naver.com/main/clusterArticles.nav...,사회
4,"경찰국 인선 마무리 행안부 ""경찰자문위 회의록 없다""",http://news.naver.com/main/clusterArticles.nav...,사회
5,"김용진 경기도 경제부지사 사임 ""도의회 파행 멈춰야""",http://news.naver.com/main/clusterArticles.nav...,사회
6,"""만 5세"" 초등 입학 학제 개편 반대 초교입학 철회하라",http://news.naver.com/main/clusterArticles.nav...,사회
7,전장연 지하철 시위 재개 5·9호선 지연 운행,http://news.naver.com/main/clusterArticles.nav...,사회
8,"이마트 포토존 논란 ""죽은 상어와 사진 찍으세요""",http://news.naver.com/main/clusterArticles.nav...,사회
9,사무처 간부 23명 자리교체 서울시의회,http://news.naver.com/main/clusterArticles.nav...,사회


## 추출하려하는 모든 섹션에 적용

In [76]:
# 모든 섹션의 토픽을 하나의 데이터프레임에 저장하기위해 빈 데이터 프레임 생셩
dict_sub={}
dict_sub['topic']=[]
dict_sub['URL']=[]
dict_sub["section"]=[]

topic_df=pd.DataFrame(dict_sub)
topic_df

,topic,URL,section


In [79]:
# 섹션별 헤드라인 추출 :
for i in range(1,7):
    temp=pd.DataFrame(get_topic(df_menu['link'][i],df_menu['section'][i]))
    topic_df=pd.concat([topic_df,temp],axis=0,ignore_index=True)
topic_df

에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생


,topic,URL,section
0,전국 7개 지방국제공항 청주공항 국제선 운항 재개,http://news.naver.com/main/clusterArticles.nav...,경제
1,최태원 SK 회장 바이든 대통령 26일 화상면담,http://news.naver.com/main/clusterArticles.nav...,경제
2,렌터카 빌렸다가 취소하니 위약금 폭탄 6~7월 제주서 多,http://news.naver.com/main/clusterArticles.nav...,경제
3,수출입은행장에 윤희성 전 부행장 첫 내부 출신,http://news.naver.com/main/clusterArticles.nav...,경제
4,현대제철 2분기 영업익 8221억원 전년比 51% 증가,http://news.naver.com/main/clusterArticles.nav...,경제
...,...,...,...
105,상반기 LNG 수출 1위 美 우크라전쟁 반사이익,http://news.naver.com/main/clusterArticles.nav...,세계
106,폭염 속 알프스가 녹는다 역대급 빙점고도 초긴장,http://news.naver.com/main/clusterArticles.nav...,세계
107,"""아베 경호"" 실패 日 경찰 이번엔 부인 탑승 경호차 추돌",http://news.naver.com/main/clusterArticles.nav...,세계
108,"러 ""유럽행 가스관"" 추가 차단 노르트스트림1 또 유지보수",http://news.naver.com/main/clusterArticles.nav...,세계


In [80]:
topic_df.head()
topic_df.tail()

,topic,URL,section
105,상반기 LNG 수출 1위 美 우크라전쟁 반사이익,http://news.naver.com/main/clusterArticles.nav...,세계
106,폭염 속 알프스가 녹는다 역대급 빙점고도 초긴장,http://news.naver.com/main/clusterArticles.nav...,세계
107,"""아베 경호"" 실패 日 경찰 이번엔 부인 탑승 경호차 추돌",http://news.naver.com/main/clusterArticles.nav...,세계
108,"러 ""유럽행 가스관"" 추가 차단 노르트스트림1 또 유지보수",http://news.naver.com/main/clusterArticles.nav...,세계
109,베이징서 인니 대통령과 회담 칩거 끝낸 시진핑,http://news.naver.com/main/clusterArticles.nav...,세계


### 위 결과에는 정치 섹션이 들어오지 않음 : 태그변경을 통해 정치섹선 추출
- 정치 섹션도 포함되게 함수 변경
- https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100은 토픽이 없음

In [26]:
# 정치의 토픽도 출력
# 함수구성
def get_topic(url,section):
    # 수집내용 list에 저장
    topic_title=[]
    topic_link=[]
    
    # 소스 요청해서 가져오기
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
    res=requests.get(url,headers=headers)
    
    # 파서기 생성(bs4 객체 생성)
    html=res.text
    bs_obj=BeautifulSoup(html,'html.parser')
    
    # sectton이 정치인 경우엔 다른 태그를 추출
    if section=='정치':
        topic_list=bs_obj.findAll('div',{'class':'cluster_foot_inner'})
        
    else:
        # 토픽 추출
        # h2태그의 class::cluster_head_topic
        topic_list=bs_obj.findAll('h2',{'class':'cluster_head_topic'})

    # 최종 수집항목을 list에 저장
    # link에 기본 url 추가
    # link에 기본 url 추가
    base_url='http://news.naver.com'
    # 전체 토픽 title과 link 출력
    for topic in topic_list:
        try:
            a_tag=topic.find('a')
            topic_title.append(a_tag.text.replace('\n',''))
            topic_link.append(base_url+a_tag['href'])
        except:
            print('에러발생')
    return({'topic':topic_title,'URL':topic_link,'section':section})

In [27]:
# 모든 섹션의 토픽을 하나의 데이터프레임에 저장하기위해 빈 데이터 프레임 생셩
dict_sub={}
dict_sub['topic']=[]
dict_sub['URL']=[]
dict_sub["section"]=[]

topic_df=pd.DataFrame(dict_sub)
topic_df

# 섹션별 헤드라인 추출 :
for i in range(1,7):
    temp=pd.DataFrame(get_topic(df_menu['link'][i],df_menu['section'][i]))
    topic_df=pd.concat([topic_df,temp],axis=0,ignore_index=True)
topic_df.head()

에러발생
에러발생
에러발생
에러발생
에러발생
에러발생
에러발생


,topic,URL,section
0,45개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
1,44개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
2,33개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
3,51개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
4,62개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치


In [28]:
# 파일로 저장
topic_df.to_csv('crawl_data/naver_news_topic.csv')

In [29]:
pd.read_csv('crawl_data/naver_news_topic.csv',index_col=0)

,topic,URL,section
0,45개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
1,44개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
2,33개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
3,51개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
4,62개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
...,...,...,...
65,하향 안정화되는 농산물 가격 글로벌 인플레 진정될까,http://news.naver.com/main/clusterArticles.nav...,세계
66,아시아 최고 여성 부호는 이재용보다 재산 더 많네,http://news.naver.com/main/clusterArticles.nav...,세계
67,출범 후 최저 日기시다 내각 지지율 58%,http://news.naver.com/main/clusterArticles.nav...,세계
68,흑인 여배우 위상 높여 스타트렉 니셸 니컬스 별세,http://news.naver.com/main/clusterArticles.nav...,세계


In [30]:
topic_df

,topic,URL,section
0,45개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
1,44개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
2,33개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
3,51개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
4,62개의 관련뉴스 더보기,http://news.naver.com/main/clusterArticles.nav...,정치
...,...,...,...
65,하향 안정화되는 농산물 가격 글로벌 인플레 진정될까,http://news.naver.com/main/clusterArticles.nav...,세계
66,아시아 최고 여성 부호는 이재용보다 재산 더 많네,http://news.naver.com/main/clusterArticles.nav...,세계
67,출범 후 최저 日기시다 내각 지지율 58%,http://news.naver.com/main/clusterArticles.nav...,세계
68,흑인 여배우 위상 높여 스타트렉 니셸 니컬스 별세,http://news.naver.com/main/clusterArticles.nav...,세계
